`Rasmus Utz Faber`

`Martín Omil Nogales`

# Unit 3 - Part 2: Trained CNNs



## 1. Selection of models, data extraction and preprocessing

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np

# Step 1: Load and Preprocess Data
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Extract a validation set from the training set (10%)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)


2025-03-16 00:02:33.905847: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-16 00:02:33.921500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742079753.938576 3521788 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742079753.943759 3521788 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-16 00:02:33.961179: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:

# Resize images to 224x224 to match most pretrained models
IMG_SIZE = (128, 128)

def resize_images(images):
    images_resized = np.array([tf.image.resize(image, IMG_SIZE).numpy() for image in images])
    return images_resized

x_train = resize_images(x_train)
x_val = resize_images(x_val)
x_test = resize_images(x_test)

# Data Augmentation Layer
# data_augmentation = keras.Sequential([
#     keras.layers.RandomFlip("horizontal"),
#     keras.layers.RandomRotation(0.2),
#     keras.layers.RandomZoom(0.2),
#     keras.layers.RandomTranslation(0.1, 0.1)
# ])

I0000 00:00:1742079759.833325 3521788 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13536 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:98:00.0, compute capability: 7.5


## 2. Feature Extraction

In [3]:
# Step 2: Feature Extraction (Initial Setup)
def build_feature_extractor(model_name):
    base_model = None

    if model_name == 'ResNet50V2':
        base_model = keras.applications.ResNet50V2(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
    elif model_name == 'VGG19':
        base_model = keras.applications.VGG19(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
    elif model_name == 'MobileNet':
        base_model = keras.applications.MobileNet(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
    elif model_name == 'InceptionV3':
        base_model = keras.applications.InceptionV3(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
    elif model_name == 'ResNet101V2':
        base_model = keras.applications.ResNet101V2(weights="imagenet", include_top=False, input_shape=(128, 128, 3))

    base_model.trainable = False  # Freeze the pretrained model

    model = keras.Sequential([
        #data_augmentation,  # Applying Data Augmentation
        base_model,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(100, activation='softmax')  # CIFAR-100 has 100 classes
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [4]:
# Step 2: Training the models for Feature Extraction

def train_model(model_name):
    model = build_feature_extractor(model_name)
    print(f"\nTraining {model_name} with Feature Extraction...")

    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=3,
        batch_size=128
    )

    # Save the trained model
    model.save(f'{model_name}_feature_extractor.keras')
    print(f"Model saved as {model_name}_feature_extractor.keras")
    
    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f"{model_name} Test Accuracy: {test_acc * 100:.2f}%")

    return history, test_acc

## 3. Fine Tuning

In [5]:
# Step 3: Fine-Tuning the models

def fine_tune_model(model_name):
    model = keras.models.load_model(f'{model_name}_feature_extractor.keras')

    # Unfreeze some layers (last 2 blocks)
    base_model = model.layers[0]
    base_model.trainable = True

    for layer in base_model.layers[:-30]:  # Keep most layers frozen
        layer.trainable = False

    # Compile the model with a low learning rate
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    print(f"\nFine-tuning {model_name}...")

    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=3,
        batch_size=128
    )

    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f"{model_name} Fine-Tuning Test Accuracy: {test_acc * 100:.2f}%")

    return history, test_acc



## 4. Training

In [ ]:
# Models to train and fine-tune
model_names = ['ResNet50V2', 'InceptionV3'] #, 'MobileNet']

results = {}

for model_name in model_names:
    # Feature Extraction
    history, test_acc = train_model(model_name)
    results[f'{model_name} (Feature Extraction)'] = test_acc

    # Fine-Tuning
    history, test_acc = fine_tune_model(model_name)
    results[f'{model_name} (Fine-Tuning)'] = test_acc

print("\nAll Results:", results)



Training ResNet50V2 with Feature Extraction...
Epoch 1/3


I0000 00:00:1742079809.991877 3521829 service.cc:148] XLA service 0x152754013360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742079809.991903 3521829 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-03-16 00:03:30.120814: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742079811.017513 3521829 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-16 00:03:35.203852: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.80GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


  2/352 ━━━━━━━━━━━━━━━━━━━━ 28s 82ms/step - accuracy: 0.0117 - loss: 6.0978   

I0000 00:00:1742079817.120826 3521829 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


351/352 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.2914 - loss: 3.1241

...

352/352 ━━━━━━━━━━━━━━━━━━━━ 59s 133ms/step - accuracy: 0.2920 - loss: 3.1207 - val_accuracy: 0.5228 - val_loss: 1.7635
Epoch 2/3
352/352 ━━━━━━━━━━━━━━━━━━━━ 62s 96ms/step - accuracy: 0.5164 - loss: 1.8043 - val_accuracy: 0.5478 - val_loss: 1.6177
Epoch 3/3
352/352 ━━━━━━━━━━━━━━━━━━━━ 41s 97ms/step - accuracy: 0.5678 - loss: 1.5488 - val_accuracy: 0.5560 - val_loss: 1.6176
Model saved as ResNet50V2_feature_extractor.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 33ms/step - accuracy: 0.5554 - loss: 1.5931
ResNet50V2 Test Accuracy: 55.75%

Fine-tuning ResNet50V2...
Epoch 1/3


2025-03-16 00:06:52.518831: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


351/352 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.5113 - loss: 1.8663

2025-03-16 00:07:42.904339: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.52GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


352/352 ━━━━━━━━━━━━━━━━━━━━ 76s 178ms/step - accuracy: 0.5115 - loss: 1.8653 - val_accuracy: 0.5678 - val_loss: 1.5810
Epoch 2/3
352/352 ━━━━━━━━━━━━━━━━━━━━ 62s 140ms/step - accuracy: 0.6256 - loss: 1.3252 - val_accuracy: 0.5876 - val_loss: 1.5000
Epoch 3/3
113/352 ━━━━━━━━━━━━━━━━━━━━ 29s 124ms/step - accuracy: 0.6726 - loss: 1.1367

## 5. Graph

In [ ]:
import matplotlib.pyplot as plt

# Generate Performance Comparison Graph
labels = list(results.keys())
scores = list(results.values())

plt.figure(figsize=(12, 6))
plt.barh(labels, scores, color='skyblue')
plt.xlabel('Test Accuracy')
plt.title('Performance Comparison of Pretrained Models on CIFAR-100')
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd

# Generate Performance Comparison Table
results_df = pd.DataFrame(list(results.items()), columns=['Model and Strategy', 'Test Accuracy'])

print("\nPerformance Comparison Table:\n")
print(results_df.to_string(index=False))